In [1]:
from __future__ import print_function

import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.activations import relu
from tensorflow.keras.regularizers import l2
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import cifar100, cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print("Packages Loaded")

Packages Loaded


In [2]:
# Now import Cifar-10 data and process it.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [ ]:
# Now import Cifar-100 data and process it.
(, t), (, ) = cifar100.load_data()
# Fill in the rest.

In [ ]:
# Our baseline model for this lab.
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(3072,)))

model.add(Dense(64, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

# Notice that you can reshape the data for an MLP inside the fit call without changing it globally!
model.fit(x_train.reshape(50000, 3072), y_train, epochs=20, batch_size=64, validation_data=(x_test.reshape(10000, 3072), y_test))

In [ ]:
# Do the same for CIFAR-100.


In [ ]:
# Create the CIFAR-10 CNN baseline here.


In [ ]:
# Create the CIFAR-100 CNN baseline here.


In [ ]:
# Change the Convolutional layers to 64 filters for CIFAR-10.


In [ ]:
# Change the Convolutional layers to 16 filters for CIFAR-10.


In [ ]:
# Remove the Max Pooling Layers for CIFAR-10.


In [ ]:
# Remove the Max Pooling Layers for CIFAR-100.


In [3]:
# Data Augmentation.
batch_size = 64
num_classes = 10
epochs = 40
data_augmentation = True

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0., 
        horizontal_flip=True,
        vertical_flip=False, 
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    datagen.fit(x_train)

    model.fit(datagen.flow(x_train, y_train,
                        batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test))

scores = model.evaluate(x_test, y_test, verbose=0)

Using real-time data augmentation.
  ...
    to  
  ['...']
Train for 782 steps, validate on 10000 samples
Epoch 1/40
782/782 [==============================] - 97s 124ms/step - loss: 1.8367 - accuracy: 0.3264 - val_loss: 1.3982 - val_accuracy: 0.4920
Epoch 2/40
782/782 [==============================] - 94s 120ms/step - loss: 1.4202 - accuracy: 0.4860 - val_loss: 1.1817 - val_accuracy: 0.5753
Epoch 3/40
782/782 [==============================] - 94s 120ms/step - loss: 1.2574 - accuracy: 0.5479 - val_loss: 1.0560 - val_accuracy: 0.6257
Epoch 4/40
782/782 [==============================] - 94s 120ms/step - loss: 1.1465 - accuracy: 0.5897 - val_loss: 1.0090 - val_accuracy: 0.6400
Epoch 5/40
782/782 [==============================] - 101s 129ms/step - loss: 1.0788 - accuracy: 0.6171 - val_loss: 0.8934 - val_accuracy: 0.6844
Epoch 6/40
782/782 [==============================] - 96s 122ms/step - loss: 1.0210 - accuracy: 0.6411 - val_loss: 0.9206 - val_accuracy: 0.6845
Epoch 7/40
782/782 [==

In [ ]:
# Here is our CIFAR-10 CNN ResNet.
inputs = tf.keras.Input(shape=(32, 32, 3), name='img')
x = Conv2D(32, 3, activation='relu')(inputs)
x = Conv2D(64, 3, activation='relu')(x)
block_1_output = MaxPooling2D(3)(x)

x = Conv2D(64, 3, activation='relu', padding='same')(block_1_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_2_output = tf.keras.layers.add([x, block_1_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_2_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_3_output = tf.keras.layers.add([x, block_2_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_3_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_4_output = tf.keras.layers.add([x, block_3_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_4_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_5_output = tf.keras.layers.add([x, block_4_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_5_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_6_output = tf.keras.layers.add([x, block_5_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_6_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_7_output = tf.keras.layers.add([x, block_6_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_7_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_8_output = tf.keras.layers.add([x, block_7_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_8_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_9_output = tf.keras.layers.add([x, block_8_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_9_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_10_output = tf.keras.layers.add([x, block_9_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_10_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_11_output = tf.keras.layers.add([x, block_10_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_11_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_12_output = tf.keras.layers.add([x, block_11_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_12_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_13_output = tf.keras.layers.add([x, block_12_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_13_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_14_output = tf.keras.layers.add([x, block_13_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_14_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_15_output = tf.keras.layers.add([x, block_14_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_15_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_16_output = tf.keras.layers.add([x, block_15_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_16_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_17_output = tf.keras.layers.add([x, block_16_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_17_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_18_output = tf.keras.layers.add([x, block_17_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_18_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_19_output = tf.keras.layers.add([x, block_18_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_19_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_20_output = tf.keras.layers.add([x, block_19_output])

x = Conv2D(64, 3, activation='relu')(block_20_output)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs, name='resnet')


model.compile(Adam(amsgrad=True), 'binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=256,
          epochs=10,
          validation_split=0.2)

In [ ]:
# Create the CIFAR-100 CNN ResNet.


In [ ]:
# Create your own CNN that rivals a CNN ResNet for CIFAR-100.
